# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 95.7%


In [5]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.93,0.98,0.93,0.96,0.90,0.98,0.98,0.98,0.94,0.95,0.98,0.98,0.98,0.98,0.98,0.98,0.94,0.98,0.98,0.98,0.98,0.92,0.97,0.98,0.98,0.98,0.98,0.94,0.95,0.94,0.94,0.91,0.96,0.98,0.98,0.96,0.98,0.95,0.98,0.95,0.98,0.93,0.99,0.98,0.93,0.92,0.90,0.96,0.98,0.88,0.92,0.98,0.98,0.98,0.98,0.87,0.97,0.98,0.98,0.98,0.98,0.92,0.98,0.91,0.98,0.98,0.92,0.98,0.98,0.98,0.96,0.94,0.98,0.91,0.98,0.98,0.90,0.91,0.97,0.82


In [6]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.99
2004,0.96
2005,0.93
2006,0.92
2007,0.95
2008,0.97
2009,0.95
2010,0.96
2011,0.96


In [7]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.95
3,0.95
4,0.94
5,0.95
6,0.95
7,0.96
8,0.97
9,0.96


In [8]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.99
     3          1.00
     4          0.98
     5          0.99
     6          1.00
     7          0.99
     8          0.98
     9          0.98
     10         0.98
     11         0.98
     12         0.98
2004 1          0.95
     2          0.94
     3          0.94
     4          0.94
     5          0.95
     6          0.95
     7          0.96
     8          0.97
     9          0.97
     10         0.97
     11         0.97
     12         0.97

## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

**Question: Are AEP calculations and losses computed for the capacity factor?** Might want to specify either way.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [11]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [12]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.45,0.47,0.44,0.46,0.43,0.47,0.47,0.47,0.45,0.46,0.47,0.47,0.47,0.47,0.47,0.47,0.45,0.47,0.47,0.46,0.46,0.44,0.47,0.47,0.47,0.47,0.46,0.45,0.46,0.45,0.45,0.43,0.46,0.47,0.47,0.46,0.47,0.46,0.47,0.45,0.47,0.44,0.47,0.47,0.44,0.45,0.43,0.46,0.47,0.41,0.44,0.46,0.47,0.47,0.47,0.41,0.46,0.47,0.47,0.47,0.47,0.44,0.47,0.43,0.47,0.47,0.45,0.47,0.47,0.47,0.46,0.46,0.46,0.44,0.47,0.47,0.43,0.43,0.46,0.38


In [13]:
# BUG: I cannot make this cell execute.
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

ValueError: Must pass 2-d input. shape=(2, 87672, 1)

In [14]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

ValueError: Must pass 2-d input. shape=(2, 87672, 1)

In [15]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

ValueError: Must pass 2-d input. shape=(2, 87672, 1)

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [16]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 99%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [17]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

ValueError: Must pass 2-d input. shape=(2, 8698, 1)

In [18]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

ValueError: Must pass 2-d input. shape=(2, 8698, 1)

In [19]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

ValueError: Must pass 2-d input. shape=(2, 8698, 1)

## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [20]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $482,444.40/MW


In [21]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","2,660,000.00","93,946,875.00"


In [22]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"10,433,781.25"
2004,"11,529,687.50"
2005,"11,585,375.00"
2006,"11,836,937.50"
2007,"13,232,250.00"
2008,"10,490,625.00"
2009,"11,727,562.50"
2010,"12,049,437.50"
2011,"12,463,500.00"


In [23]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,010.42","543,083.33","543,083.33",0.00,0.00
2,"496,125.00","495,177.08","496,052.08",0.00,0.00
3,"541,625.00","542,864.58","541,989.58",0.00,0.00
4,"525,000.00","524,635.42","524,635.42",0.00,0.00
5,"542,500.00","542,500.00","542,500.00",0.00,"30,193,750.00"
6,"525,000.00","525,000.00","525,000.00","2,660,000.00",0.00
7,"542,500.00","542,500.00","542,500.00",0.00,"31,403,125.00"
8,"542,500.00","542,500.00","542,500.00",0.00,0.00
9,"525,000.00","525,000.00","525,000.00",0.00,"32,350,000.00"


In [24]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   50,750.00               50,750.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       2,750,000.00  
     6                   52,500.00            266,000.00               0.00  
     7                   54,250.00                  0.00       2,750,000.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00       2,750,000.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00  
2004 1                   54,250.00                  0.00               0.00  
     2                   50,750.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       3,373,437.50  
     6                   52,500.00            266,000.00               0.00  
     7                   54,250.00                  0.00       3,218,750.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00       2,750,000.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00

## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [25]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.76,0.76,0.75,0.27,0.55


In [26]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.64,0.63,0.62,0.10,0.48
2004,0.91,0.90,0.91,0.10,0.64
2005,0.79,0.76,0.79,0.10,0.79
2006,0.73,0.74,0.73,0.45,0.67
2007,0.77,0.76,0.79,0.45,0.60
2008,0.77,0.78,0.78,0.17,0.44
2009,0.79,0.79,0.78,0.24,0.56
2010,0.74,0.77,0.72,0.17,0.64
2011,0.72,0.73,0.70,0.34,0.52


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

**Question: All these labor calculations are $0.** Is this intended?

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,263,023.44"
Repair,"20,782,932.29"
Crew Transfer,"1,541,958.34"
Site Travel,0.00
Mobilization,"15,000,000.00"
Weather Delay,"24,637,721.35"
No Requests,"22,332,236.98"
Not in Shift,"29,228,783.85"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,263,023.44","2,263,023.44"
Repair,0.00,0.00,0.00,"20,782,932.29","20,782,932.29"
Crew Transfer,0.00,0.00,0.00,"1,541,958.34","1,541,958.34"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"15,000,000.00","15,000,000.00"
Weather Delay,0.00,0.00,0.00,"24,637,721.35","24,637,721.35"
No Requests,0.00,0.00,0.00,"22,332,236.98","22,332,236.98"
Not in Shift,0.00,0.00,0.00,"29,228,783.85","29,228,783.85"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance      56,875.00
     Repair        1,590,145.83
     Crew Transfer   109,411.46
     Site Travel           0.00
     Mobilization  1,500,000.00
     Weather Delay 1,025,010.42
     No Requests   4,338,518.23
     Not in Shift  1,813,820.31
2004 Maintenance     440,216.15
     Repair        2,246,270.83

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                   
1     Maintenance    158,247.40
      Repair         228,411.46
      Crew Transfer   37,770.83
      Site Travel          0.00
      Mobilization         0.00
      Weather Delay  225,567.71
      No Requests    600,304.69
      Not in Shift   378,875.00
2     Maintenance    173,742.19
      Repair         198,151.04

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          20,526.04
           Crew Transfer    2,570.31
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   25,156.25
           No Requests    105,492.19
           Not in Shift    10,536.46
     2     Maintenance          0.00
           Repair          23,096.35
           Crew Transfer    2,789.06
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   11,593.75
           No Requests     99,695.31
           Not in Shift     9,825.52
     3     Maintenance          0.00
           Repair          20,598.96
           Crew Transfer    2,406.25
           Site Travel          0.00

## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
generator,"114,100,763.03"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
generator delay                  0.00              0.00   53,677,348.96   
          maintenance            0.00              0.00    2,263,023.44   
          repair                 0.00              0.00   20,782,932.29   

                         total_cost  
component action                     
generator delay       53,677,348.96  
          maintenance  2,263,023.44  
          repair      20,782,932.29

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,generator,"2,207,500.00",0.00,"4,589,356.77","6,796,856.77"
2004,generator,"4,726,000.00",0.00,"8,115,304.69","12,841,304.69"
2005,generator,"4,526,000.00",0.00,"9,139,122.40","13,665,122.40"
2006,generator,"4,211,000.00",0.00,"9,274,346.36","13,485,346.36"
2007,generator,"4,269,500.00",0.00,"10,812,502.61","15,082,002.61"
2008,generator,"3,014,000.00",0.00,"5,725,674.48","8,739,674.48"
2009,generator,"3,329,500.00",0.00,"9,103,020.83","12,432,520.83"
2010,generator,"3,931,000.00",0.00,"9,432,713.54","13,363,713.54"
2011,generator,"3,497,000.00",0.00,"9,827,557.29","13,324,557.29"


In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,generator,"1,150,000.00",0.00,"1,021,216.15","2,171,216.15"
2,generator,"1,089,500.00",0.00,"899,937.50","1,989,437.50"
3,generator,"1,146,000.00",0.00,"880,924.48","2,026,924.48"
4,generator,"1,585,000.00",0.00,"986,143.23","2,571,143.23"
5,generator,"8,304,500.00",0.00,"23,947,562.50","32,252,062.50"
6,generator,"3,608,500.00",0.00,"1,645,351.56","5,253,851.56"
7,generator,"7,708,000.00",0.00,"21,322,335.94","29,030,335.94"
8,generator,"1,196,000.00",0.00,"774,684.90","1,970,684.90"
9,generator,"6,558,000.00",0.00,"24,091,578.13","30,649,578.13"


In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     generator       35,500.00              0.00       57,257.81   
     2     generator       39,500.00              0.00       47,304.69   
     3     generator       50,000.00              0.00       31,828.13   
     4     generator       50,000.00              0.00       29,312.50   
     5     generator      380,000.00              0.00    1,268,916.67   
     6     generator       93,500.00              0.00       53,062.50   
     7     generator      375,000.00              0.00      851,651.04   
     8     generator       51,000.00              0.00       31,208.33   
     9     generator      732,000.00              0.00    1,915,408.85   
     10    generator      106,000.00              0.00      107,625.00   
     11    generator      141,500.00              0.00       89,796.88   
     12    generator      153,500.00              0.00      105,984.38   
2004 1     generator      204,000.00              0.00      162,750.00   
     2     generator      136,000.00              0.00      151,375.00   
     3     generator      224,500.00              0.00      162,750.00   
     4     generator      271,000.00              0.00      156,625.00   
     5     generator    1,267,000.00              0.00    3,034,437.50   
     6     generator      348,500.00              0.00      175,854.17   
     7     generator      890,500.00              0.00    2,845,861.98   
     8     generator      168,000.00              0.00      121,971.35   
     9     generator      534,500.00              0.00      826,750.00   
     10    generator      296,500.00              0.00      159,304.69   
     11    generator      184,500.00              0.00      155,750.00   
     12    generator      201,000.00              0.00      161,875.00   
2005 1     generator      130,000.00              0.00      162,750.00   
     2     generator      161,000.00              0.00      132,234.38   
     3     generator      111,500.00              0.00       89,304.69   
     4     generator       90,000.00              0.00       74,229.17   

                       total_cost  
year month component               
2003 1     generator    92,757.81  
     2     generator    86,804.69  
     3     generator    81,828.13  
     4     generator    79,312.50  
     5     generator 1,648,916.67  
     6     generator   146,562.50  
     7     generator 1,226,651.04  
     8     generator    82,208.33  
     9     generator 2,647,408.85  
     10    generator   213,625.00  
     11    generator   231,296.88  
     12    generator   259,484.38  
2004 1     generator   366,750.00  
     2     generator   287,375.00  
     3     generator   387,250.00  
     4     generator   427,625.00  
     5     generator 4,301,437.50  
     6     generator   524,354.17  
     7     generator 3,736,361.98  
     8     generator   289,971.35  
     9     generator 1,361,250.00  
     10    generator   455,804.69  
     11    generator   340,250.00  
     12    generator   362,875.00  
2005 1     generator   292,750.00  
     2     generator   293,234.38  
     3     generator   200,804.69  
     4     generator   164,229.17

## Fixed Cost Impacts

**Question: Are all of these costs supposed to be 0?**

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [42]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [43]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,000,080.00"


In [45]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [46]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,000,080.00"


In [47]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [48]:
# Project totals at the project level
total = metrics.process_times()
total

/Users/akey/Projects/WOMBAT/WOMBAT/wombat/core/post_processor.py:1304: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(


,time_to_completion,process_time,downtime,N
category,,,,
annual service,"217,768.95","74,130.50","67,217.25",522.00
major repair,"134,973.67","1,679.00","1,147.00",32.00
major replacement,"197,981.61","10,647.50","197,981.61",56.00
manual reset,"187,132.20","36,827.00","14,127.75","5,635.00"
medium repair,"11,427.29","8,969.50","6,819.75",187.00
minor repair,"66,801.69","31,667.75","15,254.25","2,266.00"


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [49]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

/Users/akey/Projects/WOMBAT/WOMBAT/wombat/core/post_processor.py:1366: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  production = self.production.sum(axis=0)[col_filter]


,windfarm
Project Energy Production (kWh),"9,604,316,592.50"


In [50]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

/Users/akey/Projects/WOMBAT/WOMBAT/wombat/core/post_processor.py:1366: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  production = self.production.sum(axis=0)[col_filter]


,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,604,316,592.50","117,057,719.50","122,655,697.50","116,909,499.00","120,808,688.00","112,093,732.50","123,738,684.00","123,031,689.00","122,665,615.00","118,097,049.00","120,439,419.50","123,966,170.00","122,875,480.00","123,055,380.50","123,522,342.00","123,463,523.50","123,168,825.50","117,951,151.00","122,898,401.00","122,822,404.50","121,915,973.00","121,911,834.50","115,270,990.00","122,700,379.00","123,445,764.50","123,065,299.50","122,879,469.00","121,953,092.50","118,088,984.00","119,813,673.50","119,592,184.00","117,946,408.50","112,385,147.50","121,215,242.00","123,516,060.50","122,828,349.50","120,830,853.00","122,413,104.50","119,747,459.50","122,919,382.50","118,971,797.50","123,701,344.00","115,106,780.50","124,341,740.00","123,171,076.00","115,546,175.00","117,151,870.50","113,049,701.00","120,104,918.00","122,946,224.50","108,449,681.50","114,715,115.00","122,069,824.50","123,468,864.50","123,813,550.00","122,762,876.50","107,545,014.00","120,928,719.00","123,743,960.00","123,827,461.50","123,131,629.50","122,510,780.00","114,920,849.00","122,680,277.50","113,599,849.00","122,504,736.50","122,734,260.00","117,398,047.00","122,852,468.00","122,825,444.50","123,463,711.00","120,083,639.00","119,692,712.00","122,150,410.00","116,319,620.50","123,454,299.00","122,344,287.50","113,468,174.50","113,820,180.00","122,051,595.50","101,231,807.00"


In [51]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"871,759,814.00"
2004,"937,789,546.00"
2005,"964,767,728.50"
2006,"910,596,607.00"
2007,"1,013,696,524.00"
2008,"1,066,229,717.50"
2009,"944,319,919.00"
2010,"854,689,689.00"
2011,"1,024,826,565.50"


In [52]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,072,208,941.50"
2,"809,581,025.00"
3,"886,860,678.00"
4,"694,750,770.00"
5,"676,884,662.00"
6,"552,824,707.00"
7,"588,399,365.00"
8,"644,840,787.00"
9,"796,147,569.50"


In [53]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     112,264,087.50
     2      59,460,085.50
     3      67,736,666.00
     4      92,155,108.50
     5      62,187,172.00
     6      53,973,212.00
     7      53,799,749.50
     8      52,699,139.50
     9      47,421,649.50
     10     76,890,822.50
     11     95,053,151.50
     12     98,118,970.00
2004 1      95,877,068.00
     2      82,846,560.50
     3      88,272,658.00
     4      65,571,357.50
     5      51,578,884.50
     6      64,727,159.00
     7      47,513,085.00
     8      72,675,234.50
     9     100,109,670.00
     10     98,803,163.00
     11     82,652,380.00
     12     87,162,326.00

## PySAM-Powered Results

The PySAM library is utilized for a number of project financial metrics.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [54]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [55]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [56]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [57]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [58]:
SAM_settings = "SAM_Singleowner_defaults.yaml"
metrics.sam_settings = load_yaml(sim.env.data_dir / "windfarm", SAM_settings)
metrics._setup_pysam()

Exception: singleowner execution error.
	exec fail(singleowner): PPA price from which to calculate parasitic load costs is not specified. Check inputs for Revenue and Electricity Purchases.



In [59]:
metrics.pysam_all_outputs()

Exception: Runtime error: get_cf_project_return_aftertax_npv called for SAM_Singleowner but "cf_project_return_aftertax_npv" not assigned


In [60]:
sim.env.cleanup_log_files(log_only=False)